# Crafting worst-case inital model parameters through pre-training

In [114]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import torchvision.transforms as transforms
from tqdm import tqdm
import os
from copy import deepcopy

from models import Models
from utils.data import load_data
from audit_model import test_model

device = 'cpu'

## MNIST
### Pre-train on half of MNIST

In [115]:
# hyper-parameters
data_name = 'mnist'
lr = 0.01
n_epochs = 5
batch_size = 32

# reproducibility
np.random.seed(0)
torch.manual_seed(0)

In [116]:
# load full dataset
X, y, out_dim = load_data(data_name, None, device=device, split='train')
X_test, y_test, _ = load_data(data_name, None, device=device, split='test')

len(X)

60000

In [83]:
# use only first half of dataset for pre-training
X_train, y_train = X[:len(X)//2], y[:len(X)//2]

len(X_train)

30000

In [118]:
# define model
model = Models['cnn'](X_train.shape, out_dim).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr)

In [119]:
# train model
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=False)

pbar = tqdm(range(n_epochs))
losses = []
save_model_epochs = [1, 2, 3, 4]
saved_models = []
for curr_epoch in pbar:
    for curr_X, curr_y in train_loader:
        optimizer.zero_grad()

        output = model(curr_X)
        loss = criterion(output, curr_y)
        loss.backward()

        optimizer.step()

        losses.append(loss.cpu().item())
        pbar.set_postfix({'loss': losses[-1]})
    
    if curr_epoch in save_model_epochs:
        saved_models.append(deepcopy(model)) 

100%|██████████| 5/5 [00:48<00:00,  9.68s/it, loss=0.265] 


In [120]:
model.load_state_dict(torch.load('pretrained_models/cnn_mnist_half.pt'))

/var/folders/yb/bwm5pv5j0ms2gv7c21gj70k00000gn/T/ipykernel_97016/2421651748.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('pretrained_

<All keys matched successfully>

In [121]:
# test accuracy
test_acc = test_model(model, X_test, y_test) * 100
print(f'Test accuracy (%): {test_acc:.3f}')

Test accuracy (%): 92.990


In [122]:
import os

# make both directories if missing
os.makedirs('pretrained_models', exist_ok=True)
os.makedirs('pretrained_models/cnn_mnist_half_epochs', exist_ok=True)

torch.save(model.cpu().state_dict(), 'pretrained_models/cnn_mnist_half.pt')
for epoch, m in zip(save_model_epochs, saved_models):
    torch.save(m.cpu().state_dict(),
               f'pretrained_models/cnn_mnist_half_epochs/{epoch}epochs.pt')

In [123]:
# save model
torch.save(model.cpu().state_dict(), f'pretrained_models/cnn_mnist_half.pt')
for i, (save_model_epoch, model) in enumerate(zip(save_model_epochs, saved_models)):
    torch.save(model.cpu().state_dict(), f'pretrained_models/cnn_mnist_half_epochs/{save_model_epoch}epochs.pt')

In [124]:
# save remaining half to ensure no overlap
folder = f'data/{data_name}_finetune_half/'
os.makedirs(folder, exist_ok=True)

X_finetune, y_finetune = X[len(X)//2:], y[len(y)//2:]

np.save(f'{folder}/X_train.npy', X_finetune.cpu().numpy())
np.save(f'{folder}/y_train.npy', y_finetune.cpu().numpy())
np.save(f'{folder}/X_test.npy', X_test.cpu().numpy())
np.save(f'{folder}/y_test.npy', y_test.cpu().numpy())

IGNORE ALL CIFAR 10 for NOW Deleted for now

##CONTINUE HERE


In [125]:
"""Auditing DP-SGD in black-box setting"""
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import os
import numpy as np
import argparse
from opacus.accountants.utils import get_noise_multiplier
import copy
from torch.utils.data import TensorDataset, DataLoader
import dill

from models import Models
from utils.data import load_data
from utils.dpsgd import clip_and_accum_grads
from utils.audit import compute_eps_lower_from_mia
from utils.clipbkd import craft_clipbkd, choose_worstcase_label

def xavier_init_model(model):
    """Initialize model using Xavier initialization"""
    def init_weights(m):
        if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
            torch.nn.init.xavier_normal_(m.weight)
            m.bias.data.fill_(0.01)

    model.apply(init_weights)



In [126]:
def train_model(model_name, X, y, epsilon, delta, max_grad_norm, n_epochs, lr, device='cpu', init_model=None, block_size=1024, out_dim=10):
    """Train model w/ DP-SGD (no sub-sampling + gradients are summed instead of averaged)"""
    # initialize model, loss function, and optimizer
    if init_model is None:
        model = Models[model_name](X.shape, out_dim=out_dim).to(device)
        xavier_init_model(model)
    else:
        model = copy.deepcopy(init_model)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr)
    
    # set noise level
    if epsilon is not None:
        # no subsampling, i.e., sample rate = 1
        noise_multiplier = get_noise_multiplier(target_epsilon=epsilon, target_delta=delta, sample_rate=1,
            epochs=n_epochs, accountant='prv')
    else:
        noise_multiplier = 0
    
    # train model for n_epochs
    grad_norms = []
    for epoch in tqdm(range(n_epochs), leave=False):
        optimizer.zero_grad()

        accum_grad, curr_grad_norms = clip_and_accum_grads(model, X, y, optimizer, criterion, max_grad_norm, block_size=block_size)
        if epoch == 0:
            # save per-sample gradient norms from first epoch
            grad_norms.append(curr_grad_norms)

        # accumulate per-sample gradients and add noise
        with torch.no_grad():
            for name, param in model.named_parameters():
                curr_grad = accum_grad[name]

                if noise_multiplier > 0 and max_grad_norm is not None:
                    # add noise
                    curr_grad = curr_grad + noise_multiplier * max_grad_norm * torch.randn_like(curr_grad)
                
                # update gradient of parameter
                param.grad = curr_grad
        
        # update parameter
        optimizer.step()
    
    return model, grad_norms

In [127]:

def test_model(model, X, y, batch_size=128):
    """Test trained model on test set"""
    test_loader = DataLoader(TensorDataset(X, y), batch_size=batch_size, shuffle=False)

    model.eval()
    acc = 0
    with torch.no_grad():
        for curr_X, curr_y in test_loader:
            curr_y_hat = torch.argmax(model(curr_X), dim=1)
            acc += torch.sum(curr_y_hat == curr_y).cpu().item()
    model.train()
    
    return acc / len(y)


In [128]:
def save_checkpoint(out_folder, outputs, losses, all_grad_norms, train_set_accs, test_set_accs, fit_world_only, save_grad_norms):
    """Save checkpoint"""
    # create folder if not exists
    os.makedirs(out_folder, exist_ok=True)

    # save random state
    random_state = {
        'np': np.random.get_state(),
        'torch': torch.random.get_rng_state()
    }
    dill.dump(random_state, open(f'{out_folder}/random_state.dill', 'wb'))

    # save intermediate values
    if fit_world_only:
        np.save(f'{out_folder}/outputs_{fit_world_only}.npy', outputs[fit_world_only])
        np.save(f'{out_folder}/losses_{fit_world_only}.npy', losses[fit_world_only])
        if save_grad_norms:
            np.save(f'{out_folder}/all_grad_norms_{fit_world_only}.npy', all_grad_norms[fit_world_only])

        if fit_world_only == 'out':
            np.save(f'{out_folder}/train_set_accs.npy', train_set_accs)
            np.save(f'{out_folder}/test_set_accs.npy', test_set_accs)
    else:
        np.save(f'{out_folder}/outputs_in.npy', outputs['in'])
        np.save(f'{out_folder}/outputs_out.npy', outputs['out'])
        np.save(f'{out_folder}/train_set_accs.npy', train_set_accs)
        np.save(f'{out_folder}/test_set_accs.npy', test_set_accs)
        np.save(f'{out_folder}/losses_in.npy', losses['in'])
        np.save(f'{out_folder}/losses_out.npy', losses['out'])
        if save_grad_norms:
            np.save(f'{out_folder}/all_grad_norms_in.npy', all_grad_norms['in'])
            np.save(f'{out_folder}/all_grad_norms_out.npy', all_grad_norms['out'])

In [129]:
def resume_checkpoint(out_folder, save_grad_norms, fit_world_only, resume):
    """Load checkpoint if resume is set to True and previous checkpoint exists, else create new empty checkpoint"""
    outputs = {'out': [], 'in': []}
    losses = {'out': [], 'in': []}
    all_grad_norms = { 'out': [], 'in': [] }
    train_set_accs = []
    test_set_accs = []

    if os.path.exists(out_folder) and resume:
        # if folder exists and resume is set to true load previous values
        random_state = dill.load(open(f'{out_folder}/random_state.dill', 'rb'))
        np.random.set_state(random_state['np'])
        torch.random.set_rng_state(random_state['torch'])

        if fit_world_only:
            outputs[fit_world_only] = np.load(f'{out_folder}/outputs_{fit_world_only}.npy').tolist()
            losses[fit_world_only] = np.load(f'{out_folder}/losses_{fit_world_only}.npy').tolist()
            if save_grad_norms:
                all_grad_norms[fit_world_only] = np.load(f'{out_folder}/all_grad_norms_{fit_world_only}.npy').tolist()

            if fit_world_only == 'out':
                train_set_accs = np.load(f'{out_folder}/train_set_accs.npy').tolist()
                test_set_accs = np.load(f'{out_folder}/test_set_accs.npy').tolist()
        else:
            outputs['in'] = np.load(f'{out_folder}/outputs_in.npy').tolist()
            outputs['out'] = np.load(f'{out_folder}/outputs_out.npy').tolist()
            train_set_accs = np.load(f'{out_folder}/train_set_accs.npy').tolist()
            test_set_accs = np.load(f'{out_folder}/test_set_accs.npy').tolist()
            losses['in'] = np.load(f'{out_folder}/losses_in.npy').tolist()
            losses['out'] = np.load(f'{out_folder}/losses_out.npy').tolist()
            if save_grad_norms:
                all_grad_norms['in'] = np.load(f'{out_folder}/all_grad_norms_in.npy').tolist()
                all_grad_norms['out'] = np.load(f'{out_folder}/all_grad_norms_out.npy').tolist()
    else:
        # create folder and dump initial values in
        os.makedirs(out_folder, exist_ok=True)
        save_checkpoint(out_folder, outputs, losses, all_grad_norms, train_set_accs, test_set_accs, args.fit_world_only, args.save_grad_norms)
    
    return outputs, losses, all_grad_norms, train_set_accs, test_set_accs

In [272]:
#AUDITING
from types import SimpleNamespace
import os, torch

def _pick_device():
    # Prefer Apple GPU (MPS) on Mac, else CUDA if present, else CPU
    try:
        if hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
            return "mps"
    except Exception:
        pass
    if torch.cuda.is_available():
        return "cuda:0"
    return "cpu"

# Use worst-case init if the notebook created it; otherwise use '' to fix to a random weight vector
_fixed_init_path = "pretrained_models/mnist.pt"
_fixed_init = _fixed_init_path if os.path.exists(_fixed_init_path) else ""  # '' => fix to some random weights
# average-case Xavier init instead, set _fixed_init = None

args = SimpleNamespace(
    data_name="mnist",          # mnist | cifar10 | cifar100
    model_name="lr",            # try 'cnn' if you crafted a CNN init
    n_reps=200,                 # total models (split across IN/OUT worlds)
    n_df=0,                     # 0 => full dataset
    n_epochs=100,
    lr=0.01,
    max_grad_norm=1.0,
    epsilon=6.0,                # target ε (Opacus/PRV)
    delta=1e-2,
    target_type="clipbkd",      # 'blank' | 'clipbkd' | path to a target sample
    seed=0,
    out="exp_data",
    device=_pick_device(),      # "mps" on Apple GPU, else "cuda:0" or "cpu"
    fixed_init=_fixed_init,     # path => worst-case; '' => fixed random; None => average-case Xavier
    block_size=1000,
    resume=True,                # skip work if results are already present
    fit_world_only=None,        # 'in' or 'out' to run a single world; None = both then combine
    save_grad_norms=False,
    alpha=0.05,                 # significance for empirical ε estimate
)
target = 100
args.n_reps = 1000

args.n_epochs = 5

# reproducibility
np.random.seed(args.seed)
torch.manual_seed(args.seed)

out_folder = f'{args.out}/{args.data_name}_{args.model_name}_eps{args.epsilon}'
device = args.device if torch.cuda.is_available() else 'cpu'

# load data (define D-)
if args.n_df == 1:
    # load single data point for type safety
    X_out, y_out, out_dim = load_data(args.data_name, 1, device=device)
else:
    X_out, y_out, out_dim = load_data(args.data_name, args.n_df - 1, device=device)

init_model = None
if args.fixed_init is not None:
    init_model = Models[args.model_name](X_out.shape, out_dim=out_dim).to(device)

    if args.fixed_init == '':
        # initialize model (average-case)
        xavier_init_model(init_model)
    else:
        # load weights from path (worst-case)
        init_model.load_state_dict(torch.load(args.fixed_init))

# craft target data point (x_T, y_T)
if args.target_type == 'blank':
    # blank sample
    target_X = torch.zeros_like(X_out[[0]])
    target_y = torch.from_numpy(np.array([9])).to(device)
elif args.target_type == 'clipbkd':
    # ClipBKD sample
    target_X, target_y = craft_clipbkd(X_out, init_model, device)
elif os.path.exists(args.target_type):
    # pre-crafted target sample
    target_X = torch.from_numpy(np.load(args.target_type)).to(device)
    if init_model is not None:
        target_y =  choose_worstcase_label(init_model, target_X)
    else:
        target_y = torch.from_numpy(np.array([9])).to(device)
else:
    raise Exception(f'Target {args.target_type} not found')

# define D = D- U {(x_T, y_T)}
X_in, y_in = torch.vstack((X_out, target_X)), torch.cat((y_out, target_y))

# handle case where n_df = 1
X_out, y_out = X_out[:args.n_df - 1], y_out[:args.n_df - 1]

# load test dataset
X_test, y_test, _ = load_data(args.data_name, None, split='test', device=device)

# train M on D and D-
# resume from checkpoint
outputs, losses, all_grad_norms, train_set_accs, test_set_accs = resume_checkpoint(out_folder, args.save_grad_norms, args.fit_world_only, args.resume)
worlds = [args.fit_world_only] if args.fit_world_only else ['out', 'in']
for world in worlds:
    # set dataset according to "world"
    curr_X, curr_y = (X_out, y_out) if world == 'out' else (X_in, y_in)

    # check how many reps initially completed
    reps_completed = len(outputs[world]) 

    for rep in tqdm(range(reps_completed, args.n_reps // 2), initial=reps_completed, total=args.n_reps // 2):
        # train model
        model, grad_norms = train_model(args.model_name, curr_X, curr_y, args.epsilon, args.delta,
            args.max_grad_norm, args.n_epochs, args.lr, device=device, init_model=init_model,
            block_size=args.block_size, out_dim=out_dim)
        
        # keep track of per-sample gradient norms
        all_grad_norms[world].append(grad_norms)
        
        # get loss of model on target sample
        model.eval()
        with torch.no_grad():
            output = model(target_X)
            outputs[world].append(output[0].cpu().numpy())
            losses[world].append(-nn.CrossEntropyLoss()(output, target_y).cpu().item())
        
        # get test set accuracy from first 5 reps
        if rep < 5 and world == 'out':
            if len(X_out) > 0:
                train_set_accs.append(test_model(model, X_out, y_out))
            test_set_accs.append(test_model(model, X_test, y_test))
        
        # free CUDA memory
        del model
        torch.cuda.empty_cache()

        # save checkpoint
        save_checkpoint(out_folder, outputs, losses, all_grad_norms, train_set_accs, test_set_accs, args.fit_world_only, args.save_grad_norms)
    outputs[world] = np.array(outputs[world])

if not args.fit_world_only:
    # calculate empirical epsilon using GDP
    print("losses in:")
    print(np.size(losses['in']))
    print(losses['in'])
    print("losses out:")
    print(np.size(losses['out']))
    print(losses['out'])
    mia_scores = np.concatenate([losses['in'], losses['out']])
    mia_labels = np.concatenate([np.ones_like(losses['in']), np.zeros_like(losses['out'])])
    _, emp_eps_loss = compute_eps_lower_from_mia(mia_scores, mia_labels, args.alpha, args.delta, 'GDP', n_procs=1)

    np.save(f'{out_folder}/emp_eps_loss.npy', [emp_eps_loss])
    np.save(f'{out_folder}/mia_scores.npy', mia_scores)
    np.save(f'{out_folder}/mia_labels.npy', mia_labels)

    print(f'Theoretical eps: {args.epsilon}')
    print(f'Empirical eps: {emp_eps_loss}')

print(f'Train set accuracy: {np.mean(train_set_accs) * 100:.3f}%')
print(f'Test set accuracy: {np.mean(test_set_accs) * 100:.3f}%')


  0%|          | 0/500 [00:00<?, ?it/s]/Users/benkim/miniconda3/envs/bb_audit_dpsgd/lib/python3.10/site-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(
/Users/benkim/miniconda3/envs/bb_audit_dpsgd/lib/python3.10/site-packages/opacus/accountants/analysis/prv/prvs.py:67: RuntimeWarning: divide by zero encountered in log
  z = np.log(np.where(t > np.log(1 - q), (np.exp(t) + q - 1) / q, 1))
/Users/benkim/miniconda3/envs/bb_audit_dpsgd/lib/python3.10/site-packages/opacus/accountants/analysis/prv/prvs.py:70: RuntimeWarning: divide by zero encountered in log
  t > np.log(1 - q),
100%|██████████| 500/500 [26:23<00:00,  3.17s/it]


losses in:
500
[-3.6274635791778564, -3.095247745513916, -4.559689044952393, -4.911440849304199, -3.5922889709472656, -2.5989012718200684, -3.08661150932312, -2.9248905181884766, -4.156489372253418, -3.5111823081970215, -3.4447860717773438, -3.770193099975586, -4.858377456665039, -3.6240572929382324, -2.4062252044677734, -3.8240747451782227, -3.986875295639038, -3.5966930389404297, -3.323880195617676, -3.222585439682007, -3.176693916320801, -2.954719305038452, -2.8338987827301025, -3.966488838195801, -4.331681251525879, -3.2421083450317383, -2.7651009559631348, -3.527475357055664, -3.1450886726379395, -4.213202476501465, -3.1140334606170654, -3.586107015609741, -2.7853126525878906, -2.2553675174713135, -3.3396313190460205, -4.406693458557129, -4.391239643096924, -4.901767730712891, -3.1540584564208984, -3.478832960128784, -3.676720142364502, -2.922860622406006, -4.189453601837158, -2.7875044345855713, -4.94570255279541, -3.491466999053955, -4.333553791046143, -2.826115846633911, -4.045

  0%|          | 0/1000 [00:00<?, ?it/s]python(29689) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Theoretical eps: 6.0
Empirical eps: 5.070635808403288
Train set accuracy: 55.930%
Test set accuracy: 56.076%


In [234]:
# set the seed
# recreate the code
import sys
import tensorflow as tf1
import numpy as np
import csv
import os
import random as random

tf1.compat.v1.disable_eager_execution()  
tf = tf1.compat.v1 

np.random.seed(0); tf.set_random_seed(0); random.seed(0)

alpha     = 2      # > 1
run_num   = 1
n         = 1       # feature dimension
Lrate     = 1e-5
rescaled  = 0          # 0 or 1 (alpha-scaling in objective)


method         = 'DV_Renyi'        


epochs = 1000
mb_size = 250
#number of nodes in each hidden layer (can have more than one hidden layer)
hidden_layers = [256] 

#save estimate every SF iterations
SF = 250
#samples for estimating Df
N=1000

Q_pool = np.asarray(losses['in'],  dtype=np.float32).reshape(-1, 1)
P_pool = np.asarray(losses['out'], dtype=np.float32).reshape(-1, 1)

#mu = (np.sum(Q_pool) + np.sum(Q_pool)) / (np.size(Q_pool)+np.size(P_pool))

n = P_pool.shape[1]

    

def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1.0 / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)

#construct variables for the neural networks
def initialize_W(layers):
    W_init=[]
    num_layers = len(layers)
    for l in range(0,num_layers-1):
        W_init.append(xavier_init(size=[layers[l], layers[l+1]]))
    return W_init

def initialize_NN(layers,W_init):
    NN_W = []
    NN_b = []
    num_layers = len(layers)
    for l in range(0,num_layers-1):
        W = tf.Variable(W_init[l])
        b = tf.Variable(tf.zeros([1,layers[l+1]], dtype=tf.float32), dtype=tf.float32)
        NN_W.append(W)
        NN_b.append(b)
    return NN_W, NN_b

    
       
alpha_scaling=1.
if rescaled==1:
    alpha_scaling=alpha



#variable for Q
X = tf.placeholder(tf.float32, shape=[None, n])
#variable for P
Z = tf.placeholder(tf.float32, shape=[None, n])




layers=[n]+hidden_layers+ hidden_layers+ hidden_layers +[1]
    

W_init=initialize_W(layers)
D_W, D_b = initialize_NN(layers,W_init)
theta_D = [D_W, D_b]

  
def discriminator(x):
    num_layers = len(D_W) + 1
    
    h = x
    for l in range(0,num_layers-2):
        W = D_W[l]
        b = D_b[l]
        h = tf.nn.relu(tf.add(tf.matmul(h, W), b))
    
    W = D_W[-1]
    b = D_b[-1]
    out =  tf.matmul(h, W) + b

    
    return out      

def sample_P(N_samp):
    idx = np.random.randint(0, P_pool.shape[0], size=N_samp)
    return P_pool[idx]  # (N_samp, n)

def sample_Q(N_samp):
    idx = np.random.randint(0, Q_pool.shape[0], size=N_samp)
    return Q_pool[idx]

P_data=discriminator(Z)
Q_data=discriminator(X)

P_max=tf.reduce_max((alpha-1.0)*P_data)
Q_max=tf.reduce_max(alpha*Q_data)

#DV renyi objective
objective=alpha_scaling/(alpha-1.0)*tf.math.log(tf.reduce_mean(tf.math.exp(((alpha-1.0)*P_data-P_max)/alpha_scaling)))+1.0/(alpha-1.0)*P_max-1.0/alpha*Q_max-alpha_scaling/alpha*tf.math.log(tf.reduce_mean(tf.math.exp((alpha*Q_data-Q_max)/alpha_scaling)))

objective=alpha_scaling/(alpha-1.0)*tf.math.log(tf.reduce_mean(tf.math.exp(((alpha-1.0)*P_data-P_max)/alpha_scaling)))+1.0/(alpha-1.0)*P_max-1.0/alpha*Q_max-alpha_scaling/alpha*tf.math.log(tf.reduce_mean(tf.math.exp((alpha*Q_data-Q_max)/alpha_scaling)))


#AdamOptimizer
solver = tf.train.AdamOptimizer(learning_rate=Lrate).minimize(-objective, var_list=theta_D)



config = tf.ConfigProto(device_count={'GPU': 0})
sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())



divergence_array=np.zeros(epochs//SF+1)




#use N samples for estimation of Df
Q_plot_samples=sample_Q(N)
P_plot_samples=sample_P(N)


j=0
for it in range(epochs+1):
       
    if it>0:

            X_samples=sample_Q(mb_size)
            Z_samples=sample_P(mb_size) 

            sess.run(solver, feed_dict={X: X_samples, Z: Z_samples})
    
    if it % SF == 0:                
       
        
        X_samples=Q_plot_samples
        Z_samples=P_plot_samples
        
        
        divergence_array[j]=sess.run( objective, feed_dict={X: X_samples, Z: Z_samples})
        
        print()
        print(method)
        if rescaled==1:
            print('rescaled')
        #print('mu: {}'.format(mu))
        print('alpha: {}'.format(alpha))
        print('Iter: {}'.format(it))

        
        j=j+1
        
        

print(f"Final DV Rényi estimate D_{alpha}(P||Q): {divergence_array[-1] / alpha_scaling:.6f}")

layers_str=''
for layer_dim in hidden_layers:
    layers_str=layers_str+' '+str(layer_dim)
print()
print('Hidden Layers:'+layers_str)





DV_Renyi
alpha: 2
Iter: 0

DV_Renyi
alpha: 2
Iter: 250

DV_Renyi
alpha: 2
Iter: 500

DV_Renyi
alpha: 2
Iter: 750

DV_Renyi
alpha: 2
Iter: 1000
Final DV Rényi estimate D_2(P||Q): 1.568266

Hidden Layers: 256


In [271]:
# R_a (Q||P)


# set the seed
# recreate the code
import sys
import tensorflow as tf1
import numpy as np
import csv
import os
import random as random

tf1.compat.v1.disable_eager_execution()  
tf = tf1.compat.v1 

np.random.seed(0); tf.set_random_seed(0); random.seed(0)

alpha     = 2      # > 1
run_num   = 1
n         = 1       # feature dimension
Lrate     = 1e-5
rescaled  = 0          # 0 or 1 (alpha-scaling in objective)
mu        = 0.5        # mean shift for Q in the Gaussian example

method         = 'DV_Renyi'        


epochs = 1000
mb_size = 250
#number of nodes in each hidden layer (can have more than one hidden layer)
hidden_layers = [256] 

#save estimate every SF iterations
SF = 250
#samples for estimating Df
N=1000

Q_pool = np.asarray(losses['out'],  dtype=np.float32).reshape(-1, 1)
P_pool = np.asarray(losses['in'], dtype=np.float32).reshape(-1, 1)

#mu = (np.sum(Q_pool) + np.sum(Q_pool)) / (np.size(Q_pool)+np.size(P_pool))

n = P_pool.shape[1]

    

def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1.0 / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)

#construct variables for the neural networks
def initialize_W(layers):
    W_init=[]
    num_layers = len(layers)
    for l in range(0,num_layers-1):
        W_init.append(xavier_init(size=[layers[l], layers[l+1]]))
    return W_init

def initialize_NN(layers,W_init):
    NN_W = []
    NN_b = []
    num_layers = len(layers)
    for l in range(0,num_layers-1):
        W = tf.Variable(W_init[l])
        b = tf.Variable(tf.zeros([1,layers[l+1]], dtype=tf.float32), dtype=tf.float32)
        NN_W.append(W)
        NN_b.append(b)
    return NN_W, NN_b

    
       
alpha_scaling=1.
if rescaled==1:
    alpha_scaling=alpha



#variable for Q
X = tf.placeholder(tf.float32, shape=[None, n])
#variable for P
Z = tf.placeholder(tf.float32, shape=[None, n])




layers=[n]+hidden_layers+ hidden_layers +[1]
    

W_init=initialize_W(layers)
D_W, D_b = initialize_NN(layers,W_init)
theta_D = [D_W, D_b]

  
def discriminator(x):
    num_layers = len(D_W) + 1
    
    h = x
    for l in range(0,num_layers-2):
        W = D_W[l]
        b = D_b[l]
        h = tf.nn.relu(tf.add(tf.matmul(h, W), b))
    
    W = D_W[-1]
    b = D_b[-1]
    out =  tf.matmul(h, W) + b

    
    return out      

def sample_P(N_samp):
    idx = np.random.randint(0, P_pool.shape[0], size=N_samp)
    return P_pool[idx]  # (N_samp, n)

def sample_Q(N_samp):
    idx = np.random.randint(0, Q_pool.shape[0], size=N_samp)
    return Q_pool[idx]

P_data=discriminator(Z)
Q_data=discriminator(X)

P_max=tf.reduce_max((alpha-1.0)*P_data)
Q_max=tf.reduce_max(alpha*Q_data)

#DV renyi objective
objective=alpha_scaling/(alpha-1.0)*tf.math.log(tf.reduce_mean(tf.math.exp(((alpha-1.0)*P_data-P_max)/alpha_scaling)))+1.0/(alpha-1.0)*P_max-1.0/alpha*Q_max-alpha_scaling/alpha*tf.math.log(tf.reduce_mean(tf.math.exp((alpha*Q_data-Q_max)/alpha_scaling)))

objective=alpha_scaling/(alpha-1.0)*tf.math.log(tf.reduce_mean(tf.math.exp(((alpha-1.0)*P_data-P_max)/alpha_scaling)))+1.0/(alpha-1.0)*P_max-1.0/alpha*Q_max-alpha_scaling/alpha*tf.math.log(tf.reduce_mean(tf.math.exp((alpha*Q_data-Q_max)/alpha_scaling)))


#AdamOptimizer
solver = tf.train.AdamOptimizer(learning_rate=Lrate).minimize(-objective, var_list=theta_D)



config = tf.ConfigProto(device_count={'GPU': 0})
sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())



divergence_array=np.zeros(epochs//SF+1)




#use N samples for estimation of Df
Q_plot_samples=sample_Q(N)
P_plot_samples=sample_P(N)


j=0
for it in range(epochs+1):
       
    if it>0:

            X_samples=sample_Q(mb_size)
            Z_samples=sample_P(mb_size) 

            sess.run(solver, feed_dict={X: X_samples, Z: Z_samples})
    
    if it % SF == 0:                
       
        
        X_samples=Q_plot_samples
        Z_samples=P_plot_samples
        
        
        divergence_array[j]=sess.run( objective, feed_dict={X: X_samples, Z: Z_samples})
        
        print()
        print(method)
        if rescaled==1:
            print('rescaled')
        #print('mu: {}'.format(mu))
        print('alpha: {}'.format(alpha))
        print('Iter: {}'.format(it))

        
        j=j+1
        
        

print(f"Final DV Rényi estimate D_{alpha}(P||Q): {divergence_array[-1] / alpha_scaling:.6f}")

layers_str=''
for layer_dim in hidden_layers:
    layers_str=layers_str+' '+str(layer_dim)
print()
print('Hidden Layers:'+layers_str)





DV_Renyi
alpha: 2
Iter: 0

DV_Renyi
alpha: 2
Iter: 250

DV_Renyi
alpha: 2
Iter: 500

DV_Renyi
alpha: 2
Iter: 750

DV_Renyi
alpha: 2
Iter: 1000
Final DV Rényi estimate D_2(P||Q): 0.002048

Hidden Layers: 256


Paper below to create the code experiemnts

In [ ]:
# RUNNING PAPERS EXPERIEMNTS



# -*- coding: utf-8 -*-

#Author: Jeremiah Birrell

#Neural estimation of the Renyi divergences between two n-dimensional Gaussians via 2 different methods
#1) DV method from Variational Representations and Neural Network Estimation for Renyi Divergences,  SIAM Journal on Mathematics of Data Science}, 2021
#2) CC method from Function-space regularized Renyi divergences, ICLR 2023

import sys
import tensorflow as tf1
import numpy as np
import csv
import os
tf1.compat.v1.disable_eager_execution()  
tf1.compat.v1.reset_default_graph()
tf = tf1.compat.v1 

#alpha  run_num  n  Lrate  method  IC_activation  rescaled  mu

# alpha  run_num  n   Lrate  method    act   rescaled  rho
sys.argv = ['renyi_gaussian.py', '0.5', '1', '40', '2e-4', 'DV_Renyi', 'relu', '0', '0.8']


alpha_str =sys.argv[1]
alpha=float(alpha_str)  # alpha>1
run_num=int(sys.argv[2])    # run number
n=int(sys.argv[3]) #dimension of gaussians
Lrate=float(sys.argv[4])    # learning rate
method=sys.argv[5] #DV_Renyi,inf_conv_Renyi
IC_activation=sys.argv[6] #abs, elu, relu, polysoftplus
rescaled=int(sys.argv[7]) #1 if use rescaled Renyi
mu_str=sys.argv[8]
mu=float(mu_str) #mean of Q distriubtion

rho = float(mu_str)   
mu  = 0.0


epochs = 10000
mb_size = 1000
#number of nodes in each hidden layer (can have more than one hidden layer)
hidden_layers = [256] 



#save estimate every SF iterations
SF = 100
#samples for estimating Df
N=50000





#means, and covariances of the Gaussian r.v.s
mu_p=np.zeros((n,1))
mu_q=np.zeros((n,1))
#mu_q[0]=mu



Sigma_p=np.identity(n)
Sigma_q=np.identity(n)



#M@np.transpose(M)=Sigma
Mp=np.linalg.cholesky(Sigma_p)
Mq=np.linalg.cholesky(Sigma_q)
    


def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1.0 / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)


#construct variables for the neural networks
def initialize_W(layers):
    W_init=[]
    num_layers = len(layers)
    for l in range(0,num_layers-1):
        W_init.append(xavier_init(size=[layers[l], layers[l+1]]))
    return W_init

def initialize_NN(layers,W_init):
    NN_W = []
    NN_b = []
    num_layers = len(layers)
    for l in range(0,num_layers-1):
        W = tf.Variable(W_init[l])
        b = tf.Variable(tf.zeros([1,layers[l+1]], dtype=tf.float32), dtype=tf.float32)
        NN_W.append(W)
        NN_b.append(b)
    return NN_W, NN_b



        
Sigma_alpha=alpha*Sigma_q+(1.0-alpha)*Sigma_p       
       
alpha_scaling=1.
if rescaled==1:
    alpha_scaling=alpha


#ADDED
d = n // 2
I = np.eye(d)
Sigma_p = np.block([[I, rho*I],[rho*I, I]])  
Sigma_q = np.eye(2*d)                        
mu_p = np.zeros((2*d,1)); mu_q = np.zeros((2*d,1))


#Exact R_alpha(P||Q)
Renyi_exact=1.0/2.0*np.matmul(np.transpose(mu_q-mu_p),np.matmul(np.linalg.inv(Sigma_alpha),mu_q-mu_p))-1.0/(2.0*alpha*(alpha-1.0))*np.math.log(np.linalg.det(Sigma_alpha)/(np.math.pow(np.linalg.det(Sigma_p),1.0-alpha)*np.math.pow(np.linalg.det(Sigma_q),alpha)))
divergence_exact=alpha_scaling*Renyi_exact[0][0]




#variable for Q
X = tf.placeholder(tf.float32, shape=[None, n])
#variable for P
Z = tf.placeholder(tf.float32, shape=[None, n])




layers=[n]+hidden_layers+hidden_layers +[1]
    

W_init=initialize_W(layers)
D_W, D_b = initialize_NN(layers,W_init)
theta_D = [D_W, D_b]

  
def discriminator(x):
    num_layers = len(D_W) + 1
    
    h = x
    for l in range(0,num_layers-2):
        W = D_W[l]
        b = D_b[l]
        h = tf.nn.relu(tf.add(tf.matmul(h, W), b))
    
    W = D_W[-1]
    b = D_b[-1]
    out =  tf.matmul(h, W) + b

    
    return out        
 

#IC_activation: abs, elu, relu, polysoftplus

if IC_activation=='abs':
    def IC_final_layer(y):
        return -tf.math.abs(y)
elif IC_activation=='elu':
    def IC_final_layer(y):
        return -(tf.nn.elu(y)+1.)
elif IC_activation=='relu':
    def IC_final_layer(y):
        return -tf.nn.relu(y)
elif IC_activation=='polysoftplus':
    def IC_final_layer(y):
        return -(1.+(1./(1.+tf.nn.relu(-y))-1)*(1.-tf.sign(y))/2. +y*(tf.sign(y)+1.)/2. )

# def sample_P(N_samp):
#     return np.transpose((mu_p+np.matmul(Mp,np.random.normal(0., 1.0, size=[n, N_samp]))))

# def sample_Q(N_samp):
#     return np.transpose((mu_q+np.matmul(Mq,np.random.normal(0., 1.0, size=[n, N_samp]))))

# JOINT VS PRODUCT
def sample_P(N_samp):
    d = n // 2
    X   = np.random.normal(0., 1.0, size=[N_samp, d])
    eps = np.random.normal(0., 1.0, size=[N_samp, d])
    Y   = rho * X + np.sqrt(1 - rho**2) * eps
    return np.hstack([X, Y])

def sample_Q(N_samp):
    d = n // 2
    X = np.random.normal(0., 1.0, size=[N_samp, d])
    Y = np.random.normal(0., 1.0, size=[N_samp, d])
    return np.hstack([X, Y])

P_data=discriminator(Z)
Q_data=discriminator(X)

P_max=tf.reduce_max((alpha-1.0)*P_data)
Q_max=tf.reduce_max(alpha*Q_data)

if method=='DV_Renyi':
    objective=alpha_scaling/(alpha-1.0)*tf.math.log(tf.reduce_mean(tf.math.exp(((alpha-1.0)*P_data-P_max)/alpha_scaling)))+1.0/(alpha-1.0)*P_max-1.0/alpha*Q_max-alpha_scaling/alpha*tf.math.log(tf.reduce_mean(tf.math.exp((alpha*Q_data-Q_max)/alpha_scaling)))
elif method=='inf_conv_Renyi':
    objective=tf.reduce_mean(IC_final_layer(Q_data))+alpha_scaling/(alpha-1.)*tf.math.log(tf.reduce_mean(tf.math.pow(-IC_final_layer(P_data)/alpha_scaling,(alpha-1.)/alpha)))+(alpha_scaling/alpha)*(np.math.log(alpha)+1.)



#AdamOptimizer
solver = tf.train.AdamOptimizer(learning_rate=Lrate).minimize(-objective, var_list=theta_D)



config = tf.ConfigProto(device_count={'GPU': 0})
sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())



divergence_array=np.zeros(epochs//SF+1)




#use N samples for estimation of Df
Q_plot_samples=sample_Q(N)
P_plot_samples=sample_P(N)


j=0
for it in range(epochs+1):
       
    if it>0:

            X_samples=sample_Q(mb_size)
            Z_samples=sample_P(mb_size) 

            sess.run(solver, feed_dict={X: X_samples, Z: Z_samples})
    
    if it % SF == 0:                
       
        
        X_samples=Q_plot_samples
        Z_samples=P_plot_samples
        
        
        divergence_array[j]=sess.run( objective, feed_dict={X: X_samples, Z: Z_samples})
        
        print()
        print(method)
        if rescaled==1 and not(method=='alpha_div'):
            print('rescaled')
        if method=='inf_conv_Renyi':
            print(IC_activation)
        print('mu: {}'.format(mu))
        print('alpha: {}'.format(alpha))
        print('Iter: {}'.format(it))
        print('Divergence: {:.6f}'.format(divergence_exact))
        print('Divergence est.: {:.6f}'.format(divergence_array[j]))
        print('Error: {:.6f}'.format(divergence_array[j]-divergence_exact))
        print('Rel. err.: {:.6f}'.format(1.0-divergence_array[j]/divergence_exact))
        
        j=j+1
        
    
        
        
rel_err=1.-divergence_array/divergence_exact

layers_str=''
for layer_dim in hidden_layers:
    layers_str=layers_str+' '+str(layer_dim)
print()
print('Hidden Layers:'+layers_str)

test_name=method+'_Gaussian_est/'
if method=='inf_conv_Renyi':
    test_name=IC_activation+'_'+test_name
if rescaled==1:
    test_name='rescaled_'+test_name

if not os.path.exists(test_name):
    os.makedirs(test_name)
    
with open(test_name+'Exact_divergence_alpha_'+alpha_str+'_mu_'+mu_str+'_Lrate_{:.1e}'.format(Lrate)+'_epochs_'+str(epochs)+'_mbsize_'+str(mb_size)+'_dim_'+str(n)+'_layers_'+layers_str+'run'+str(run_num)+'.csv', "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerow([divergence_exact])  

with open(test_name+'Divergence_est_alpha_'+alpha_str+'_mu_'+mu_str+'_Lrate_{:.1e}'.format(Lrate)+'_epochs_'+str(epochs)+'_mbsize_'+str(mb_size)+'_dim_'+str(n)+'_layers_'+layers_str+'run'+str(run_num)+'.csv', "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerow(divergence_array)  

with open(test_name+'rel_err_alpha_'+alpha_str+'_mu_'+mu_str+'_Lrate_{:.1e}'.format(Lrate)+'_epochs_'+str(epochs)+'_mbsize_'+str(mb_size)+'_dim_'+str(n)+'_layers_'+layers_str+'run'+str(run_num)+'.csv', "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerow(rel_err)  

In [236]:
import numpy as np

def mle_gaussian(X):
    """
    Maximum-likelihood estimates for a (uni/multi)variate Gaussian.

    Args:
        X: array-like, shape (n, d) or (n,) — samples

    Returns:
        mu:  shape (d,)                    (MLE mean)
        Sigma: shape (d, d)                (MLE covariance, bias=True i.e., /n)
    """
    X = np.asarray(X, dtype=np.float64)
    if X.ndim == 1:
        X = X.reshape(-1, 1)
    n, d = X.shape
    if n < 1:
        raise ValueError("Need at least one sample.")

    mu = X.mean(axis=0)
    Xc = X - mu
    Sigma = (Xc.T @ Xc) / n   # MLE: divide by n (not n-1)
    return mu, Sigma


def _logdet_psd(A, jitter=1e-12):
    """Stable log|A| for SPD/PSD matrices by adding a tiny jitter if needed."""
    A = np.asarray(A, dtype=np.float64)
    # Symmetrize to reduce numerical asymmetry
    A = 0.5 * (A + A.T)
    # Try Cholesky; fall back to eig if needed
    try:
        L = np.linalg.cholesky(A + jitter * np.eye(A.shape[0]))
        return 2.0 * np.sum(np.log(np.diag(L)))
    except np.linalg.LinAlgError:
        w = np.linalg.eigvalsh(A + jitter * np.eye(A.shape[0]))
        if np.any(w <= 0):
            raise np.linalg.LinAlgError("Covariance not positive definite.")
        return np.sum(np.log(w))


def renyi_gaussian(mu_q, Sigma_q, mu_p, Sigma_p, alpha, jitter=1e-12):
    """
    Closed-form Rényi divergence R_α(Q || P) between Gaussians, matching the
    paper's definition R_α = (1/(α(α-1))) log ∫ q^α p^{1-α}.

    Args:
        mu_q, mu_p: shape (d,)
        Sigma_q, Sigma_p: shape (d, d)
        alpha: float, α>0, α≠1
        jitter: small ridge added to Σ_α for numerical stability

    Returns:
        R_alpha: float
    """
    if alpha <= 0 or np.isclose(alpha, 1.0):
        raise ValueError("alpha must satisfy α>0 and α≠1.")

    mu_q = np.atleast_1d(mu_q).astype(np.float64)
    mu_p = np.atleast_1d(mu_p).astype(np.float64)
    Sigma_q = np.atleast_2d(Sigma_q).astype(np.float64)
    Sigma_p = np.atleast_2d(Sigma_p).astype(np.float64)

    dmu = (mu_p - mu_q).reshape(-1, 1)
    Sigma_alpha = (1.0 - alpha) * Sigma_p + alpha * Sigma_q

    # Stabilize Σ_α for inversion/logdet
    Sigma_alpha = 0.5 * (Sigma_alpha + Sigma_alpha.T) + jitter * np.eye(Sigma_alpha.shape[0])

    # log ∫ q^α p^{1-α} dx  for Gaussians
    # ln I_α = -0.5[ α(1-α) Δμᵀ Σ_α^{-1} Δμ + ln|Σ_α| - α ln|Σ_q| - (1-α) ln|Σ_p| ]
    inv_Sa = np.linalg.inv(Sigma_alpha)
    quad = float(dmu.T @ inv_Sa @ dmu)  # scalar
    ln_I = -0.5 * (alpha * (1.0 - alpha) * quad
                   + _logdet_psd(Sigma_alpha)
                   - alpha * _logdet_psd(Sigma_q)
                   - (1.0 - alpha) * _logdet_psd(Sigma_p))

    # Paper's convention: R_α = (1/(α(α-1))) * ln I_α
    return ln_I / (alpha * (alpha - 1.0))


# ---- Your data → MLEs → (optional) Rényi via plug-in ------------------------

# Given:
# Q_pool = np.asarray(losses['in'],  dtype=np.float32).reshape(-1, 1)
# P_pool = np.asarray(losses['out'], dtype=np.float32).reshape(-1, 1)

# 1) Fit Gaussians by MLE
mu_q, Sigma_q = mle_gaussian(P_pool)
mu_p, Sigma_p = mle_gaussian(Q_pool)

# 2) (Optional) Compute R_α(Q || P) with the Gaussian closed form
alpha = 10  # pick your order α>0, α≠1
R_alpha_QP = renyi_gaussian(mu_q, Sigma_q, mu_p, Sigma_p, alpha)
R_alpha_PQ = renyi_gaussian(mu_p, Sigma_p, mu_q, Sigma_q, alpha)

print("Q: mean =", mu_q.ravel()[0], " var =", float(Sigma_q))
print("P: mean =", mu_p.ravel()[0], " var =", float(Sigma_p))
print(f"R_{alpha}(Q || P) =", R_alpha_QP)
print(f"R_{alpha}(P || Q) =", R_alpha_PQ)


Q: mean = -5.543795566082001  var = 0.054147053472015
P: mean = -5.392570258617401  var = 0.04975853989793086
R_10(Q || P) = 0.12328976037942072
R_10(P || Q) = 1.1188146927223872


/var/folders/yb/bwm5pv5j0ms2gv7c21gj70k00000gn/T/ipykernel_97016/1941857340.py:74: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  quad = float(dmu.T @ inv_Sa @ dmu)  # scalar
/var/folders/yb/bwm5pv5j0ms2gv7c21gj70k00000gn/T/ipykernel_97016/1941857340.py:99: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print("Q: mean =", mu_q.ravel()[0], " var =", float(Sigma_q))
/var/folders/yb/bwm5pv5j0ms2gv7c21gj70k00000gn/T/ipykernel_97016/1941857340.py:100: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (

In [287]:
def dv_renyi_PQ():
    tf1.compat.v1.disable_eager_execution()  
    tf = tf1.compat.v1 

    #np.random.seed(0); tf.set_random_seed(0); random.seed(0)

    alpha     = 10      # > 1
    run_num   = 1
    n         = 1       # feature dimension
    Lrate     = 1e-5
    rescaled  = 0          # 0 or 1 (alpha-scaling in objective)

    method         = 'DV_Renyi'        


    epochs = 1000
    mb_size = 250
    #number of nodes in each hidden layer (can have more than one hidden layer)
    hidden_layers = [256] 

    #save estimate every SF iterations
    SF = 250
    #samples for estimating Df
    N=1000

    Q_pool = np.asarray(losses['out'],  dtype=np.float32).reshape(-1, 1)
    P_pool = np.asarray(losses['in'], dtype=np.float32).reshape(-1, 1)

    #mu = (np.sum(Q_pool) + np.sum(Q_pool)) / (np.size(Q_pool)+np.size(P_pool))

    n = P_pool.shape[1]

        

    def xavier_init(size):
        in_dim = size[0]
        xavier_stddev = 1.0 / tf.sqrt(in_dim / 2.)
        return tf.random_normal(shape=size, stddev=xavier_stddev)

    #construct variables for the neural networks
    def initialize_W(layers):
        W_init=[]
        num_layers = len(layers)
        for l in range(0,num_layers-1):
            W_init.append(xavier_init(size=[layers[l], layers[l+1]]))
        return W_init

    def initialize_NN(layers,W_init):
        NN_W = []
        NN_b = []
        num_layers = len(layers)
        for l in range(0,num_layers-1):
            W = tf.Variable(W_init[l])
            b = tf.Variable(tf.zeros([1,layers[l+1]], dtype=tf.float32), dtype=tf.float32)
            NN_W.append(W)
            NN_b.append(b)
        return NN_W, NN_b

        
        
    alpha_scaling=1.
    if rescaled==1:
        alpha_scaling=alpha



    #variable for Q
    X = tf.placeholder(tf.float32, shape=[None, n])
    #variable for P
    Z = tf.placeholder(tf.float32, shape=[None, n])




    layers=[n]+hidden_layers +[1]
        

    W_init=initialize_W(layers)
    D_W, D_b = initialize_NN(layers,W_init)
    theta_D = [D_W, D_b]

    
    def discriminator(x):
        num_layers = len(D_W) + 1
        
        h = x
        for l in range(0,num_layers-2):
            W = D_W[l]
            b = D_b[l]
            h = tf.nn.relu(tf.add(tf.matmul(h, W), b))
        
        W = D_W[-1]
        b = D_b[-1]
        out =  tf.matmul(h, W) + b

        
        return out      

    def sample_P(N_samp):
        idx = np.random.randint(0, P_pool.shape[0], size=N_samp)
        return P_pool[idx]  # (N_samp, n)

    def sample_Q(N_samp):
        idx = np.random.randint(0, Q_pool.shape[0], size=N_samp)
        return Q_pool[idx]

    P_data=discriminator(Z)
    Q_data=discriminator(X)

    P_max=tf.reduce_max((alpha-1.0)*P_data)
    Q_max=tf.reduce_max(alpha*Q_data)

    #DV renyi objective
    objective=alpha_scaling/(alpha-1.0)*tf.math.log(tf.reduce_mean(tf.math.exp(((alpha-1.0)*P_data-P_max)/alpha_scaling)))+1.0/(alpha-1.0)*P_max-1.0/alpha*Q_max-alpha_scaling/alpha*tf.math.log(tf.reduce_mean(tf.math.exp((alpha*Q_data-Q_max)/alpha_scaling)))

    objective=alpha_scaling/(alpha-1.0)*tf.math.log(tf.reduce_mean(tf.math.exp(((alpha-1.0)*P_data-P_max)/alpha_scaling)))+1.0/(alpha-1.0)*P_max-1.0/alpha*Q_max-alpha_scaling/alpha*tf.math.log(tf.reduce_mean(tf.math.exp((alpha*Q_data-Q_max)/alpha_scaling)))


    #AdamOptimizer
    solver = tf.train.AdamOptimizer(learning_rate=Lrate).minimize(-objective, var_list=theta_D)



    config = tf.ConfigProto(device_count={'GPU': 0})
    sess = tf.Session(config=config)
    sess.run(tf.global_variables_initializer())



    divergence_array=np.zeros(epochs//SF+1)




    #use N samples for estimation of Df
    Q_plot_samples=sample_Q(N)
    P_plot_samples=sample_P(N)


    j=0
    for it in range(epochs+1):
        
        if it>0:

                X_samples=sample_Q(mb_size)
                Z_samples=sample_P(mb_size) 

                sess.run(solver, feed_dict={X: X_samples, Z: Z_samples})
        
        if it % SF == 0:                
        
            
            X_samples=Q_plot_samples
            Z_samples=P_plot_samples
            
            
            divergence_array[j]=sess.run( objective, feed_dict={X: X_samples, Z: Z_samples})
            
            #print()
            #print(method)
            if rescaled==1:
                print('rescaled')
            #print('mu: {}'.format(mu))
            # print('alpha: {}'.format(alpha))
            # print('Iter: {}'.format(it))

            
            j=j+1
            
            

    print(f"Final DV Rényi estimate D_{alpha}(P||Q): {divergence_array[-1] / alpha_scaling:.6f}")

    layers_str=''
    for layer_dim in hidden_layers:
        layers_str=layers_str+' '+str(layer_dim)
    # print()
    # print('Hidden Layers:'+layers_str)
    return divergence_array[-1]

In [288]:
def dv_renyi_QP():
    tf1.compat.v1.disable_eager_execution()  
    tf = tf1.compat.v1 

    #np.random.seed(0); tf.set_random_seed(0); random.seed(0)

    alpha     = 10      # > 1
    run_num   = 1
    n         = 1       # feature dimension
    Lrate     = 1e-5
    rescaled  = 0          # 0 or 1 (alpha-scaling in objective)

    method         = 'DV_Renyi'        


    epochs = 1000
    mb_size = 250
    #number of nodes in each hidden layer (can have more than one hidden layer)
    hidden_layers = [256] 

    #save estimate every SF iterations
    SF = 250
    #samples for estimating Df
    N=1000

    Q_pool = np.asarray(losses['out'],  dtype=np.float32).reshape(-1, 1)
    P_pool = np.asarray(losses['in'], dtype=np.float32).reshape(-1, 1)

    #mu = (np.sum(Q_pool) + np.sum(Q_pool)) / (np.size(Q_pool)+np.size(P_pool))

    n = P_pool.shape[1]

        

    def xavier_init(size):
        in_dim = size[0]
        xavier_stddev = 1.0 / tf.sqrt(in_dim / 2.)
        return tf.random_normal(shape=size, stddev=xavier_stddev)

    #construct variables for the neural networks
    def initialize_W(layers):
        W_init=[]
        num_layers = len(layers)
        for l in range(0,num_layers-1):
            W_init.append(xavier_init(size=[layers[l], layers[l+1]]))
        return W_init

    def initialize_NN(layers,W_init):
        NN_W = []
        NN_b = []
        num_layers = len(layers)
        for l in range(0,num_layers-1):
            W = tf.Variable(W_init[l])
            b = tf.Variable(tf.zeros([1,layers[l+1]], dtype=tf.float32), dtype=tf.float32)
            NN_W.append(W)
            NN_b.append(b)
        return NN_W, NN_b

        
        
    alpha_scaling=1.
    if rescaled==1:
        alpha_scaling=alpha



    #variable for Q
    X = tf.placeholder(tf.float32, shape=[None, n])
    #variable for P
    Z = tf.placeholder(tf.float32, shape=[None, n])




    layers=[n]+hidden_layers+[1]
        

    W_init=initialize_W(layers)
    D_W, D_b = initialize_NN(layers,W_init)
    theta_D = [D_W, D_b]

    
    def discriminator(x):
        num_layers = len(D_W) + 1
        
        h = x
        for l in range(0,num_layers-2):
            W = D_W[l]
            b = D_b[l]
            h = tf.nn.relu(tf.add(tf.matmul(h, W), b))
        
        W = D_W[-1]
        b = D_b[-1]
        out =  tf.matmul(h, W) + b

        
        return out      

    def sample_P(N_samp):
        idx = np.random.randint(0, P_pool.shape[0], size=N_samp)
        return P_pool[idx]  # (N_samp, n)

    def sample_Q(N_samp):
        idx = np.random.randint(0, Q_pool.shape[0], size=N_samp)
        return Q_pool[idx]

    P_data=discriminator(Z)
    Q_data=discriminator(X)

    P_max=tf.reduce_max((alpha-1.0)*P_data)
    Q_max=tf.reduce_max(alpha*Q_data)

    #DV renyi objective
    objective=alpha_scaling/(alpha-1.0)*tf.math.log(tf.reduce_mean(tf.math.exp(((alpha-1.0)*P_data-P_max)/alpha_scaling)))+1.0/(alpha-1.0)*P_max-1.0/alpha*Q_max-alpha_scaling/alpha*tf.math.log(tf.reduce_mean(tf.math.exp((alpha*Q_data-Q_max)/alpha_scaling)))

    objective=alpha_scaling/(alpha-1.0)*tf.math.log(tf.reduce_mean(tf.math.exp(((alpha-1.0)*P_data-P_max)/alpha_scaling)))+1.0/(alpha-1.0)*P_max-1.0/alpha*Q_max-alpha_scaling/alpha*tf.math.log(tf.reduce_mean(tf.math.exp((alpha*Q_data-Q_max)/alpha_scaling)))


    #AdamOptimizer
    solver = tf.train.AdamOptimizer(learning_rate=Lrate).minimize(-objective, var_list=theta_D)



    config = tf.ConfigProto(device_count={'GPU': 0})
    sess = tf.Session(config=config)
    sess.run(tf.global_variables_initializer())



    divergence_array=np.zeros(epochs//SF+1)




    #use N samples for estimation of Df
    Q_plot_samples=sample_Q(N)
    P_plot_samples=sample_P(N)


    j=0
    for it in range(epochs+1):
        
        if it>0:

                X_samples=sample_Q(mb_size)
                Z_samples=sample_P(mb_size) 

                sess.run(solver, feed_dict={X: X_samples, Z: Z_samples})
        
        if it % SF == 0:                
        
            
            X_samples=Q_plot_samples
            Z_samples=P_plot_samples
            
            
            divergence_array[j]=sess.run( objective, feed_dict={X: X_samples, Z: Z_samples})
            
            #print()
            #print(method)
            if rescaled==1:
                print('rescaled')
            #print('mu: {}'.format(mu))
            # print('alpha: {}'.format(alpha))
            # print('Iter: {}'.format(it))

            
            j=j+1
            
            

    print(f"Final DV Rényi estimate D_{alpha}(P||Q): {divergence_array[-1] / alpha_scaling:.6f}")

    layers_str=''
    for layer_dim in hidden_layers:
        layers_str=layers_str+' '+str(layer_dim)
    #print()
    #print('Hidden Layers:'+layers_str)
    return divergence_array[-1]

In [289]:
renyi_measures_PQ = []
renyi_measures_QP = []
for i in range(20):
    tf1.compat.v1.reset_default_graph()  # avoid graph accumulation
    renyi_measures_PQ.append(dv_renyi_PQ())
    tf1.compat.v1.reset_default_graph()  # avoid graph accumulation
    renyi_measures_QP.append(dv_renyi_QP())

renyi_measures_PQ = np.array(renyi_measures_PQ, dtype=float)
renyi_measures_QP = np.array(renyi_measures_QP, dtype=float)

print(f"Dα(P||Q) mean: {renyi_measures_PQ.mean():.6f}")
print(f"Dα(P||Q) max : {renyi_measures_PQ.max():.6f}")
print(f"Dα(P||Q) min : {renyi_measures_PQ.min():.6f}")

print(f"Dα(Q||P) mean: {renyi_measures_QP.mean():.6f}")
print(f"Dα(Q||P) max : {renyi_measures_QP.max():.6f}")
print(f"Dα(Q||P) min : {renyi_measures_QP.min():.6f}")

def convert(measure, alpha, delta):
    return measure + np.log(1/delta)/(alpha - 1)

print(f"DP(P||Q) mean: {convert(renyi_measures_PQ.mean(), 5, 1e-2):.6f}")
print(f"DP(P||Q) max : {convert(renyi_measures_PQ.max(), 5, 1e-2):.6f}")
print(f"DP(P||Q) min : {convert(renyi_measures_PQ.min(), 5, 1e-2):.6f}")

print(f"DP(Q||P) mean: {convert(renyi_measures_QP.mean(), 5, 1e-2):.6f}")
print(f"DP(Q||P) max : {convert(renyi_measures_QP.max(), 5, 1e-2):.6f}")
print(f"DP(Q||P) min : {convert(renyi_measures_QP.min(), 5, 1e-2):.6f}")



Final DV Rényi estimate D_10(P||Q): 0.010374
Final DV Rényi estimate D_10(P||Q): 4.882215
Final DV Rényi estimate D_10(P||Q): 0.582652
Final DV Rényi estimate D_10(P||Q): -0.734646
Final DV Rényi estimate D_10(P||Q): 2.427503
Final DV Rényi estimate D_10(P||Q): 2.841002
Final DV Rényi estimate D_10(P||Q): -1.501082
Final DV Rényi estimate D_10(P||Q): 4.195619
Final DV Rényi estimate D_10(P||Q): 1.768286
Final DV Rényi estimate D_10(P||Q): 3.748070
Final DV Rényi estimate D_10(P||Q): 2.405452
Final DV Rényi estimate D_10(P||Q): 0.333911
Final DV Rényi estimate D_10(P||Q): 4.294810
Final DV Rényi estimate D_10(P||Q): 2.270423
Final DV Rényi estimate D_10(P||Q): 0.165233
Final DV Rényi estimate D_10(P||Q): -0.366689
Final DV Rényi estimate D_10(P||Q): 1.249598
Final DV Rényi estimate D_10(P||Q): -0.069278
Final DV Rényi estimate D_10(P||Q): 1.913866
Final DV Rényi estimate D_10(P||Q): 4.071554
Final DV Rényi estimate D_10(P||Q): 4.832768
Final DV Rényi estimate D_10(P||Q): 2.355656
Final 

In [290]:
print(f"DP(P||Q) mean: {convert(renyi_measures_PQ.mean(), 10, 1e-2):.6f}")
print(f"DP(P||Q) max : {convert(renyi_measures_PQ.max(), 10, 1e-2):.6f}")
print(f"DP(P||Q) min : {convert(renyi_measures_PQ.min(), 10, 1e-2):.6f}")

print(f"DP(Q||P) mean: {convert(renyi_measures_QP.mean(), 10, 1e-2):.6f}")
print(f"DP(Q||P) max : {convert(renyi_measures_QP.max(), 10, 1e-2):.6f}")
print(f"DP(Q||P) min : {convert(renyi_measures_QP.min(), 10, 1e-2):.6f}")


DP(P||Q) mean: 1.989163
DP(P||Q) max : 5.344454
DP(P||Q) min : -0.989397
DP(Q||P) mean: 2.409469
DP(Q||P) max : 5.393901
DP(Q||P) min : -0.222961
